<a href="https://colab.research.google.com/github/121deepti/Zomato-Restaurant-Clustering-and-Sentiment-Analysis/blob/main/Zomato_Restaurant_Clustering_and_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - Zomato Restaurant Clustering and Sentiment Analysis



##### **Project Type**    - Unsupervised
##### **Contribution**    - Individual

# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


**Write Problem Statement Here.**

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import missingno as msno

!pip install wordcloud
from wordcloud import WordCloud

from scipy.stats import norm

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
#for tokenization
from nltk.tokenize import word_tokenize
# for POS tagging(Part of speech in NLP sentiment analysis)
nltk.download('averaged_perceptron_tagger')


### Dataset Loading

In [ ]:
# Load Dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
hotel_df=pd.read_csv("/content/drive/MyDrive/my_data/Zomato Restaurant names and Metadata.csv")
review_df=pd.read_csv("/content/drive/MyDrive/my_data/Zomato Restaurant reviews.csv")

### Dataset First View

In [ ]:
# Restaurant Dataset First Look
hotel_df.sample(5)

In [ ]:
# Review Dataset First Look
review_df.sample(5)

### Dataset Rows & Columns count

In [ ]:
# Restaurant Dataset Rows & Columns count
hotel_df.shape

In [ ]:
# Reviews Dataset Rows & Columns count
review_df.shape

### Dataset Information

In [ ]:
# Restaurant Dataset Info
hotel_df.info()

In [ ]:
# Reviews Dataset Reviews info
review_df.info()

#### Duplicate Values

In [ ]:
# Hotel Dataset Duplicate Value Count
hotel_df.duplicated().sum()

In [ ]:
# Reviews Dataset Duplicate Value Count
review_df.duplicated().sum()

In [ ]:
#Duplicates records of Reviews Dataset
review_df[review_df.duplicated()]

In [ ]:
#Removing Duplicates from Review Dataset
review_df=review_df.drop_duplicates()
review_df.duplicated().sum()

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count of Restaurant dataset
hotel_df.isna().sum()

In [ ]:
# Missing Values/Null Values Count of User Review dataset
review_df.isna().sum()

In [ ]:
# Visualizing the missing values of Restaurant dataset
msno.bar(df=hotel_df,sort='ascending',figsize=(3,3),fontsize=8)

In [ ]:
# Visualizing the missing values of Review dataset
msno.bar(df=review_df,sort='ascending',figsize=(3,3),fontsize=8)

### What did you know about your dataset?

**Restaurant DataSet**

There are 105 total observation with 6 different features.
Feature like timing has 1 but collection has more than 50% null values.
There is no duplicate values i.e., 105 unique data.
Feature cost represent amount but has object data type because these values are separated by comma ','.
Timing represent operational hour but as it is represented in the form of text has object data type.

**Review DataSet**

There are total 10000 observation and 7 features.
Except picture and restaurant feature all others have null values.
There are total of 36 duplicate values for two restaurant - American Wild Wings and Arena Eleven, where all these duplicate values generally have null values that's why i have removed those duplicate records.
Rating represent ordinal data, has object data type should be integer.
Timing represent the time when review was posted but show object data type, it should be converted into date time.


## ***2. Understanding Your Variables***

In [ ]:
# Restaurant Dataset Columns
hotel_df.columns

In [ ]:
# Review Dataset Columns
review_df.columns

In [ ]:
# Restaurant Dataset Describe
hotel_df.describe(include='all')

In [ ]:
# Review Dataset Describe
review_df.describe(include='all')

### Variables Description

**Zomato Restaurant**

Name : Name of Restaurant

Links : URL Links of Restaurants

Cost : Per person estimated Cost of dining

Collection : Tagging of Restaurants w.r.t. Zomato categories

Cuisines : Cuisines served by Restaurants

Timings : Restaurant Timings


**Zomato Restaurant Reviews**


Restaurant : Name of the Restaurant

Reviewer : Name of the Reviewer

Review : Review Text

Rating : Rating Provided by Reviewer

MetaData : Reviewer Metadata - No. of Reviews and followers

Time: Date and Time of Review

Pictures : No. of pictures posted with review

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable of Zomato dataset.
for i in hotel_df.columns:
  col=hotel_df[i]
  print("Number of Unique values in",i,"is:",col.nunique())

In [ ]:
# Check Unique Values for each variable of Zomato Reviews dataset.
for i in review_df.columns:
  col=review_df[i]
  print("Number of Unique values in",i,"is:",col.nunique())

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.
#Creating copies of datasets for performing eda
hotel_cpy=hotel_df.copy()
review_cpy=review_df.copy()

In [ ]:
# Remove commas from the 'Cost' column and convert it to numeric
hotel_cpy['Cost'] = hotel_cpy['Cost'].str.replace(',', '').astype(int)
#Creating bins of cost column and categorize cost column
hotel_cpy['Cost_bins']=pd.cut(hotel_cpy['Cost'],bins=[100,500,1000,2000,3000])
print('[Cost per person][Number of Restaurants]')
hotel_cpy['Cost_bins'].value_counts()

In [ ]:
#top 5 costlier restaurant
hotel_cpy.sort_values('Cost', ascending = False)[['Name','Cost']][:5]

In [ ]:
#top 5 economical restaurant
hotel_cpy.sort_values('Cost')[['Name','Cost']][:5]

In [ ]:
# spliting the Collections after removing Nulls and storing unique collection in dictionary
Collections_Values=hotel_cpy['Collections'].dropna().str.split(', ')
Collections_Values_set=set(Collections_Values.explode())
collection_dict=dict.fromkeys(Collections_Values_set,0)

In [ ]:
# Updating Collection dictionary with each collection value
def collection(tag):
  tag_list=tag.split(', ')
  for t in tag_list:
    if t in collection_dict:
      collection_dict[t]+=1

In [ ]:
print(collection_dict)

In [ ]:
#Removing nulls from collections column
Collections=hotel_cpy['Collections'].dropna()

#Calling collection method
Collections.apply(collection)

#Creating data frame from dictionary
collection_df=pd.Series(collection_dict).to_frame().reset_index().rename(columns={'index':'Tags',0:'Number of Restaurants'})

#print the top 5 tags that have maximum number of restaurant have
collection_df.sort_values(by='Number of Restaurants',ascending=False)[:5]


In [ ]:
# spliting the Cuisines column and storing unique Cuisines in dictionary
Cuisines_Values=hotel_cpy['Cuisines'].str.split(', ')
Cuisines_Values_set=set(Cuisines_Values.explode())
Cuisines_dict=dict.fromkeys(Cuisines_Values_set,0)

In [ ]:
# Updating Cuisines dictionary with each Cuisines value
def Cuisines(val):
  val_list=val.split(', ')
  for t in val_list:
    if t in Cuisines_dict:
      Cuisines_dict[t]+=1

In [ ]:
#Calling Cuisines method
hotel_cpy.Cuisines.apply(Cuisines)

#Creating data frame from dictionary
cuisine_df=pd.Series(Cuisines_dict).to_frame().reset_index().rename(columns={'index':'Cuisines',0:'Number of Restaurants'})

#print the top 5 popular cuisines
cuisine_df.sort_values(by='Number of Restaurants',ascending=False)[:5]

In [ ]:
#Categorize Collections based on average cost
hotel_cpy.groupby(["Collections"])["Cost"].mean().sort_values(ascending=False).reset_index()

In [ ]:
#Categorize Cuisines based on average cost
hotel_cpy.groupby(["Cuisines"])["Cost"].mean().sort_values(ascending=False).reset_index()

In [ ]:
#Shows how many times a restaurnt got an review
review_cpy.groupby('Restaurant')['Reviewer'].count().sort_values(ascending=False).reset_index()

In [ ]:
#Replacing char value of'Like' with median value for futher analysis
median_df=review_cpy[review_cpy.Rating!='Like']
median=median_df.Rating.median()
review_cpy.Rating=review_cpy.Rating.replace('Like',median)

#Changing data type to float
review_cpy.Rating=review_cpy.Rating.astype(float)

In [ ]:
#Splitting reviewer's metadata column into total review and total follwers of that reviewer
review_cpy['Reviewer_Total_Review'],review_cpy['Reviewer_Followers']=review_cpy['Metadata'].str.split(',').str

#fetching number from the text and convert it to numeric data type
review_cpy['Reviewer_Total_Review'] = pd.to_numeric(review_cpy['Reviewer_Total_Review'].str.split(' ').str[0])
review_cpy['Reviewer_Followers'] = pd.to_numeric(review_cpy['Reviewer_Followers'].str.split(' ').str[1])

#Converting the time column to datetime type and fetch year, month from it
review_cpy['Time']=pd.to_datetime(review_cpy['Time'])
review_cpy['Review_Year'] = pd.DatetimeIndex(review_cpy['Time']).year
review_cpy['Review_Month'] = pd.DatetimeIndex(review_cpy['Time']).month

In [ ]:
#The top 10 Reviewers and their rating which have a large number of followers
review_cpy.groupby('Reviewer').agg({'Reviewer_Total_Review':max,'Reviewer_Followers':max,'Rating':'mean'}).round({'Rating': 1}).sort_values(by='Reviewer_Followers',ascending=False).head(10)

In [ ]:
#finding hotel without review
non_reviewed_hotel_list=[i for i in review_cpy.Restaurant]
non_reviewed_hotel_list=[i for i in hotel_cpy.Name if i not in non_reviewed_hotel_list]
print(non_reviewed_hotel_list)

In [ ]:
#Finding top 5 reviewers based on reviews posted
review_cpy.groupby('Reviewer')['Review'].count().sort_values(ascending=False).reset_index().head(5)

In [ ]:
#fetching which year and month are in reviewer's focus
review_cpy.groupby(['Review_Year','Review_Month'])['Review'].count().sort_values(ascending=False).reset_index().rename(columns={'Review':'Total Reviews'})

In [ ]:
#Merging the hotel and review datasets
hotel_cpy.rename(columns={'Name':'Restaurant'},inplace=True)
merged_df = hotel_cpy.merge(review_cpy,on='Restaurant')
merged_df.shape

In [ ]:
#top 5 highly rated restaurant with their cost
avg_hotel_rating=merged_df.groupby('Restaurant').agg({'Rating':'mean','Cost':'mean'}).round(1).reset_index()
avg_hotel_rating.sort_values(by='Rating',ascending=False).head(5)

In [ ]:
#top 5 low rated restaurant with their cost
avg_hotel_rating.sort_values(by='Rating').head(5)

### What all manipulations have you done and insights you found?

Firstly, I started with changing data types for cost and rating. In rating there was only one rating which was string or has value of like so I change it into median of the rating. This was done to make data consistent.

**Restaurant dataset:**first I binned cost column and found that there are around 40 restaurant in the cost range of 500-1000 and only 2 restaurant in the cost range of 2000-3000.
I also figured out 5 costlier restaurant in which Collage - Hyatt Hyderabad Gachibowli has maximum price of 2800 and then found the lowest which is Amul and Mohammedia Shawarma with price of 150.North indian cuisine is preferred choice with Great Buffet tags is mostly used in hotels.
The restaurant tag with Sunday Brunch is on the top based on price.
Continental, Italian, North Indian, Chinese, Asian cuisines restaurants have the highest cost 2800.

**Review dataset:** There are around 98 restaurants which are reviewed by 100 reviewers.After splitting reviewer's metadata column into total review and total follwers of that reviewer I found that the most followed critic which was Satwinder Singh who posted total of 186 reviews and had followers of 13410 who gives an average of 3.67 rating for each order he makes.
I have also converted the time column to datetime type and fetch year and month from it.There are some restaurants which are not reviewed at all.
Parijat Ray and Ankita posted the highest number of reviews.On May,2019 the highest number of total 1347 reviews are posted.

Then I merged the two dataset together to figure out the price point for the restaurant, top rated restaurant AB's - Absolute Barbecues has a price point of 1500 and the low rated Hotel Zara Hi-Fi has price point of 400.

In order to exactly understand why even with price point of 1500 these hotel has maximum number of rating and sentiment of those rating need to extract words from the text and do futher analysis of the review and then followed by forming clusters so that one can get recommendation about top quality restaurants.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

###**Univariate Analysis**

In [ ]:
review_df.info()

#### Chart - 1 Countplot for counting restaurants based on cost

In [ ]:
# Chart - 1 visualization code
fig=plt.figure(figsize=(10,5))
ax=fig.gca()
sns.countplot(data=hotel_df,y='Cost',order=hotel_df['Cost'].value_counts(ascending=True).index)
for p in ax.patches:
    ax.annotate(f'{int(p.get_width())}', (p.get_width(), p.get_y() + p.get_height() / 2), ha='left', va='center')
plt.show()

##### 1. Why did you pick the specific chart?

This chart helps in understanding the count of restaurants on the basis of cost

##### 2. What is/are the insight(s) found from the chart?

It shows that 13 restaurants cost is 500 and less number of expensive and cheap restaurants. It means more number of restaurants are in medium range(500-700)  

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

It signifies the medium range hotels is the choice of most customers which means this range is an affordable range for many customers.

#### Chart - 2 Different Tags associated Restaurants

In [ ]:
# Chart - 2 visualization code
fig=plt.figure(figsize=(8,5))
ax=fig.gca()
sorted=collection_df.sort_values('Number of Restaurants', ascending = False)
sns.barplot(data=sorted,y='Tags',x='Number of Restaurants')
for p in ax.patches:
    ax.annotate(f'{int(p.get_width())}', (p.get_width(), p.get_y() + p.get_height() / 2), ha='left', va='center')
plt.show()

##### 1. Why did you pick the specific chart?

This chart shows that how many restaurants are covered under the tags.  

##### 2. What is/are the insight(s) found from the chart?

Here, under Great Buffet tag comes around 220 restaurants.Likewise Food hygine rated restaurant tag is of 160 restaurants.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

This chart of tags used to describe food can help a restaurant review and food delivery platform Zomato to identify the most popular adjectives used to describe the food. This information can be used to make strategic decisions about which food attributes to focus on promoting and expanding. For example, if a significant portion of customers are describing the food as "delicious" or "fresh", Zomato could focus on adding more restaurants that are known for their delicious and fresh food and promoting them to customers.

#### Chart - 3 WordCloud for Collection

In [ ]:
# Chart - 3 visualization code
#wordcloud for tags
# storing all tags in form of text
plt.figure(figsize=(15,8))
text = " ".join(name for name in collection_df.Tags )


# Creating word_cloud with text as argument in .generate() method

word_cloud = WordCloud(width = 500, height = 500,collocations = False,
                      colormap='rainbow', background_color = 'black').generate(text)

# Display the generated Word Cloud

plt.imshow(word_cloud, interpolation='bilinear');

plt.axis("off");


##### 1. Why did you pick the specific chart?

This word cloud helps to infer the keywords from the tags.

##### 2. What is/are the insight(s) found from the chart?

Some of the words like Hyderabad,Great,Rated and Best are the high frequency words in the tags.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Similarly, a word cloud of tags used to describe food can help Zomato identify the most frequently mentioned food attributes in customer reviews. This can provide insight into which attributes are most popular and well-regarded among customers, and which attributes may need improvement.
However, it's important to note that these types of charts do not provide all the information about the business, and can not be the only decision making factor. For example, a pie chart showing that a certain adjective is popular does not tell us about the profitability of that adjective or the competition in that category. The same goes for word cloud, it only shows us the frequency of the adjective mentioned, it can not tell us if the mentions are positive or negative.

Additionally, these charts do not provide information about the other factors that can impact the business such as market trends, consumer preferences, and economic conditions. Therefore, it's important for Zomato to consider other data and information when making strategic decisions. Also, it's important to note that the data used for creating these charts should be cleaned and validated, as the results may be biased if the data is not accurate or complete.

#### Chart - 4 Cuisines offered by Restaurants

In [ ]:
# Chart - 4 visualization code
fig=plt.figure(figsize=(5,10))
ax=fig.gca()
sorted=cuisine_df.sort_values('Number of Restaurants', ascending = False)
sns.barplot(data=sorted,y='Cuisines',x='Number of Restaurants')
for p in ax.patches:
    ax.annotate(f'{int(p.get_width())}', (p.get_width(), p.get_y() + p.get_height() / 2), ha='left', va='center')
plt.show()

##### 1. Why did you pick the specific chart?

This chart tells the classification of restaurants based on their cuisines

##### 2. What is/are the insight(s) found from the chart?

Here North Indian is the most likely cuisine after that Chinese is also the choice.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

This information can be used to make strategic decisions about which cuisines to focus on promoting and expanding. For example, as the significant portion of customers are searching for north indian restaurants, Zomato could focus on adding more north indian restaurants to its platform and promoting them to customers.

#### Chart - 5 Word Cloud for Cuisines

In [ ]:
# Chart - 5 visualization code

#wordcloud for Cuisine
# storind all cuisine in form of text
plt.figure(figsize=(15,8))
text = " ".join(name for name in cuisine_df.Cuisines )


# Creating word_cloud with text as argument in .generate() method

word_cloud = WordCloud(width = 500, height = 500,collocations = False,
                      colormap='rainbow', background_color = 'black').generate(text)

# Display the generated Word Cloud

plt.imshow(word_cloud, interpolation='bilinear');

plt.axis("off");

##### 1. Why did you pick the specific chart?

This chart locates the keywords related to cuisines.

##### 2. What is/are the insight(s) found from the chart?

here I found some most frequent words like north, indian and food etc.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Similarly, a word cloud of cuisine can help Zomato identify the most frequently mentioned cuisine types in customer reviews. This can provide insight into which cuisines are most popular and well-regarded among customers, and which cuisines may need improvement.

However, these types of charts do not provide all the information about the business, and can not be the only decision making factor. For example, a pie chart showing that a certain cuisine is popular does not tell us about the profitability of that cuisine or the competition in that category. The same goes for word cloud, it only shows us the frequency of the cuisine mentioned, it can not tell us if the mentions are positive or negative.

Additionally, these charts do not provide information about the other factors that can impact the business such as market trends, consumer preferences, and economic conditions. Therefore, it's important for Zomato to consider other data and information when making strategic decisions.

#### Chart - 6 Countplot of Reviewers

In [ ]:
# Chart - 6 visualization code
fig=plt.figure(figsize=(10,5))
ax=fig.gca()
sns.countplot(data=review_cpy,y='Reviewer',order=review_df['Reviewer'].value_counts(ascending=False).index[:10],hue='Review_Year')


##### 1. Why did you pick the specific chart?

This plot finds out how many reviews are posted by which reviewer and in which year.

##### 2. What is/are the insight(s) found from the chart?

Parijat ray contribution is highest in posting reviews, others in top list are Ankita, Kiran and so on.
This chart also shows that most of the reviews are provied in the year 2018,2019

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Only count of review doesn't have any significance to business context.This should be analyzed further like rating provided by reviewer and his review.

#### Chart - 7 Distribution plot

In [ ]:
from scipy.stats import norm
plt.figure(figsize = (18,8));
for i,col in enumerate(['Cost','Rating','Review_Year']) :
    # plt.figure(figsize = (8,5));
    plt.subplot(2,2,i+1);
    sns.distplot(merged_df[col], color = '#055E85', fit = norm);
    feature = merged_df[col]
    plt.axvline(feature.mean(), color='#ff033e', linestyle='dashed', linewidth=3,label= 'mean');  #red
    plt.axvline(feature.median(), color='#A020F0', linestyle='dashed', linewidth=3,label='median'); #cyan
    plt.legend(bbox_to_anchor = (1.0, 1))
    plt.title(f'{col.title()}');
    plt.tight_layout();

##### 1. Why did you pick the specific chart?

This plot shows the distribution of numeric values.

##### 2. What is/are the insight(s) found from the chart?

All three are show skewness.
Maximum restaurant show price range for 500.
In 2018 number of reviews are more.


##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Price always place important role in any business alongwith rating which show how much engagement are made for the product.

But in this chart it is unable to figure any impact on business when plotted all alone.

###**Bivariate Analysis**

#### Chart - 8 Top10 Cheap and Costly Restaurants

In [ ]:
# Chart - 8 visualization code
fig, axes = plt.subplots(1,2,figsize=(15,3))

#Top 10 costly restaurants
sorted_df=hotel_cpy.sort_values(by='Cost',ascending=False).head(10)
sns.barplot( y='Name',x='Cost',data=sorted_df,orient='h ',ax=axes[0])
axes[0].set_title('Top 10 Costly Restaurant')

#Top 10 cheap restaurants
sorted_df1=hotel_cpy.sort_values(by='Cost').head(10)
sns.barplot( y='Name',x='Cost',data=sorted_df1,orient='h', ax=axes[1])
axes[1].set_title('Top 10 Cheap Restaurant')

plt.tight_layout()
plt.show()

##### 1. Why did you pick the specific chart?

I want to find top 10 costly and cheap restaurants available.

##### 2. What is/are the insight(s) found from the chart?

Collage-hyatt Hyderabad Gachibowli is on top of costly restaurant around 2800 cost per person while Mohammedia Shawarma is on top of cheap restaurant around 150 per person.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Cost is an important factor in business growth but only cost can't decide the popularity of the restaurant.

#### Chart - 9 High Rated v/s Low Rated Restaurant

In [ ]:
# Chart - 9 visualization code
#top 5 highly rated restaurant with their cost
fig, axes = plt.subplots(1,2,figsize=(18,5))

#Top 10 High Rated Restaurant
df=merged_df.groupby('Name').agg({'Rating':'mean','Cost':'mean'}).round(1).sort_values(by='Rating',ascending=False).reset_index().head(10)
sns.barplot(y='Name',x='Cost',data=df,orient='h',hue='Rating',ax=axes[0])
axes[0].set_title('Top 10 Best Rated Restaurant')

#Top 10 Low Rated Restaurant
df1=merged_df.groupby('Name').agg({'Rating':'mean','Cost':'mean'}).round(1).sort_values(by='Rating').reset_index().head(10)
sns.barplot(y='Name',x='Cost',data=df1,orient='h',hue='Rating',ax=axes[1])
axes[1].set_title('Top 10 Low Rated Restaurant')

plt.tight_layout()
plt.show()

##### 1. Why did you pick the specific chart?

This chart is a showcase of top 10 high/low rated restaurant's cost per person.

##### 2. What is/are the insight(s) found from the chart?

As can be analysed that AB's-Absolute Barbeques restaurant is top rated restaurant around 4.9 rating but its cost is lower than other restarunt which are lower inrating but higher in cost.
Some low rated restaurant like Club Rouge has a very high cost.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

It can be said that cost and rating is related but except cost some other factors also influence the rating like customer reviews.

#### Chart - 10 Categorization of Restaurant based on Price and Ratings

In [ ]:
# Chart - 10 visualization code
plt.figure(figsize=(16,6))
df=merged_df.groupby('Name').agg({'Rating':'mean','Cost':'mean'}).round(1).sort_values(by='Cost',ascending=False).reset_index()
sns.scatterplot(data= df, x= "Name", y="Rating", size="Cost",
                hue = 'Cost',legend=True, sizes=(20, 2000),palette ="icefire")
plt.xticks(rotation=90)
plt.title('Restaurants based on Price and Ratings',size=20,color = 'red')
plt.show()

##### 1. Why did you pick the specific chart?

This visulization depicts the relation between rating and cost of the restaurant.

##### 2. What is/are the insight(s) found from the chart?

Based on the above chart it is clear that restaurant Collage - Hyatt Hyderabad Gachibowli is most expensive restaurant in the locality which has a price of 2800 for order and has 3.5 average rating.
Hotels like Amul and Mohammedia Shawarma are least expensive with price of 150 and has 3.9 average rating.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Most expensive product are always center of attraction for a niche market (subset of the market on which a specific product is focused) at the same time for a business purpose, this product are preffered to be most revenue generating market.

Definetly for food delivery platform Zomato, it is very important to focus and improve sales based on these hotels.

Based on the average rating of 3.4 these product should increase their engagement as this may cause negative brand impact. However true behaviour can only be inspected through analysing of reviews.

#### Chart - 11 Popular Reviewers and their Ratings

In [ ]:
plt.figure(figsize=(10,6))
#creating a data frame to find most popular reviewers
df=review_cpy.groupby('Reviewer').agg({'Reviewer_Total_Review':'max','Reviewer_Followers':'max','Rating':'mean'}).round(1).sort_values(by='Reviewer_Followers',ascending=False).reset_index().head(10)

sns.scatterplot(data= df, x= "Reviewer", y="Reviewer_Followers", size="Reviewer_Total_Review",
                hue = 'Reviewer_Total_Review',legend=True, sizes=(20, 2000),style='Rating',palette ="icefire")

plt.xticks(rotation=90)
plt.title('Most Popular Critics',size=20,color = 'red')
plt.show()

##### 1. Why did you pick the specific chart?

Plotting total review, average reviewer rating, and total follower allows to see the correlation between these variables and how they relate to one another for each reviewer. It can also give insight on how reviewers with more followers tend to get more reviews, how their ratings tend to be, etc.

##### 2. What is/are the insight(s) found from the chart?

Satwinder singh is the most popular critic who has maximum number of follower and on an average he gave 180 reviews and 3.7 rating.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

This information can be used to make strategic decisions about which reviewers to focus on promoting and expanding. For example, if a certain reviewer has a high average rating and a large number of followers, Zomato could focus on promoting their reviews to customers.

It's important to note that this chart does not provide all the information about the business, and can not be the only decision making factor. However it can help on promotions food based on reviews.

#### Chart - 14 - Correlation Heatmap

In [ ]:
# Correlation Heatmap visualization code
sns.heatmap(merged_df.corr(), annot=True, cmap = 'icefire', linewidths = 1)


##### 1. Why did you pick the specific chart?

A correlation matrix is a table showing correlation coefficients between variables. Each cell in the table shows the correlation between two variables. A correlation matrix is used to summarize data, as an input into a more advanced analysis, and as a diagnostic for advanced analyses. The range of correlation is [-1,1].

Thus to know the correlation between all the variables along with the correlation coeficients, i used correlation heatmap.

##### 2. What is/are the insight(s) found from the chart?

Review_Month and Review_Year are highly negatively correlatd while Reviewer followers and Reviewer total Review are positvely correlted.
Rest all correlation can be depicted from the above chart.

#### Chart - 15 - Pair Plot

In [ ]:
# Pair Plot visualization code
sns.pairplot(merged_df)

##### 1. Why did you pick the specific chart?

Pair plot is used to understand the best set of features to explain a relationship between two variables or to form the most separated clusters. It also helps to form some simple classification models by drawing some simple lines or make linear separation in our data-set.

Thus, I used pair plot to analyse the patterns of data and realationship between the features. It's exactly same as the correlation map but here you will get the graphical representation.

##### 2. What is/are the insight(s) found from the chart?

It can be seen that there is no significant correlation between the given features in the merged dataframe.

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

Answer Here.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 2

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 3

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

In [ ]:
# Handling Missing Values & Missing Value Imputation
hotel_cpy.isna().sum()

In [ ]:
#checking null values in Collections
missing_percentage = ((hotel_cpy['Collections'].isnull().sum())/(len(hotel_cpy['Collections'])))*100
print(f'Percentage of missing value in Collections is {round(missing_percentage, 2)}%')


In [ ]:
#dropping collection column since has more than 50% of null values
hotel_cpy.drop('Collections', axis = 1, inplace = True)

#replacing Null value of Timing column with mode
mode=hotel_cpy.Timings.mode()
hotel_cpy.Timings=hotel_cpy.Timings.fillna(mode[0])

In [ ]:
hotel_cpy.isna().sum()

In [ ]:
# Handling Missing Values & Missing Value Imputation
review_cpy.isna().sum()

In [ ]:
#Checking records where Reviewer total review is Null
review_cpy[review_cpy.Reviewer_Total_Review.isna()]

In [ ]:
#removing those records where Reviewer total reviews are Nulls all values are NULLs
review_cpy=review_cpy[~(review_cpy['Reviewer_Total_Review'].isna())]

In [ ]:
#Replacing review nulls with no review
review_cpy.Review.fillna('No review',inplace=True)

#Replacing reviewer followers nulls with 0
review_cpy.Reviewer_Followers.fillna(0,inplace=True)

In [ ]:
#Finally checking the null values in review dataset
review_cpy.isna().sum()

In [ ]:
#merged the both datasets
merged=hotel_cpy.merge(review_cpy, on='Restaurant')
merged.shape

#### What all missing value imputation techniques have you used and why did you use those techniques?

Dataset that contains details about hotel, had 1 null value in timing feature and more than 50% null value in collection feature. In order to treat with those I first replaced the null value for timing with mode since there was only one null and mode is robust to outliers plus that hotel name was one unique feature which had all other feature except timing and collection so it was better to preserve that data. Since there was more than 50% null values in collection feature, I removed the entire column because columns with a high percentage of null values are likely to have a lot of missing data, which can make it difficult to accurately analyze or make predictions based on the data.

In the dataset tha has details of reviewer had Reviewer - 2, Review - 9, Rating - 2, Metadata - 2, Time - 2, Reviewer_Total_Review- 3, Reviewer_Followers - 1581, Review_Year - 2, Review_Month - 2, Review_Hour - 2. On analysing I found that feature like reviewer and reviewer total review had all null values, therefore I removed those two columns which made null values in other feature to zero except in review and reviewer followers columns. Since review was textual data, I changed those 7 null values to 'no review' and reviewer followers to 0 as follower is the meta data for reviewer and it can be 0.

And thus all the null values were treated, at the end I then again merged both the dataset hotel and review dataset.



### 2. Handling Outliers

In [ ]:
# Handling Outliers & Outlier treatments
plt.figure(figsize=(12,8))
merged.boxplot()

In [ ]:
merged.describe()

As we can see from the above analysis Reviewer Followers and Reviewer total reviews have a lot of outliers present.In Cost, there are also 2 outliers are present.
From the stastical context, we can say that there is huge difference is present between max and 75% that signifies the presence of outliers.

In [ ]:
# Handling Outliers & Outlier treatments
outlier_cols=['Cost','Reviewer_Total_Review','Reviewer_Followers']
def clip_outliers(hotel_df):
    for col in hotel_df[outlier_cols]:
        # using IQR method to define range of upper and lower limit.
        q1 = hotel_df[col].quantile(0.25)
        q3 = hotel_df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # replacing the outliers with upper and lower bound
        hotel_df[col] = hotel_df[col].clip(lower_bound, upper_bound)
    return hotel_df

In [ ]:
# using the function to treat outliers
merged = clip_outliers(merged)

In [ ]:
#BoxPlot after clipping outliers
plt.figure(figsize=(10,5))
sns.boxplot(data=merged[outlier_cols])
plt.show()

##### What all outlier treatment techniques have you used and why did you use those techniques?

We have implemented clipping method. In this method, we set a cap on our outliers data, which means that if a value is higher than or lower than a certain threshold, all values will be considered outliers. This method replaces values that fall outside of a specified range with either the minimum or maximum value within that range. Here we have set the threshold of .25-1.5IQR(Lower limit) and .75+1.5IQR(Upper limit).

### 3. Categorical Encoding

In [ ]:
# Encode your categorical columns
#create a data frame cuisine
cuisine_df=hotel_cpy[['Restaurant','Cuisines']]

#Splitting the cuisine column
cuisine_df['Cuisines']=cuisine_df.Cuisines.apply(lambda x:str(x).split(', '))

#explode the cuisines column to generate seperate value for each cuisine
cuisine_df=cuisine_df.explode('Cuisines')

#Dummification of cuisine column
cuisine_df= pd.get_dummies(cuisine_df, columns=["Cuisines"], prefix=["Cuisines"])

#grouping each restaurant as explode created unnecessary rows
cuisine_df = cuisine_df.groupby("Restaurant").sum().reset_index()


my_lambda=lambda x: 1 if x>=1 else 0
cuisine_df = cuisine_df.apply(lambda col: col.map(my_lambda) if col.name != 'Restaurant' else col)

In [ ]:
#merge cuisine dataframe with avg hotel rating to remove unrated restaurant
cluster_df=cuisine_df.merge(avg_hotel_rating,on='Restaurant')

In [ ]:
cluster_df.shape

#### What all categorical encoding techniques have you used & why did you use those techniques?

The cuisines column is a text column so first I split and explode that column to find single value of cuisine then dummify that column to convert it in numeric form .
As explode generate duplicate rows so I grouped the rows to avoid dupicacy.
finally I got 100 rows and 43 columns.
After grouping there is sum of cuisine value so I replace the sum with 1.

### 4. Textual Data Preprocessing
(It's mandatory for textual dataset i.e., NLP, Sentiment Analysis, Text Clustering etc.)

#### 1. Expand Contraction

In [ ]:
review_df.shape

In [ ]:
# Expand Contraction
!pip install contractions
import contractions

#Create a new data frame containing columns analazing the review
sentiment_df=review_cpy[['Reviewer','Review','Rating','Restaurant']]

# applying fuction for contracting text
sentiment_df['Review']=sentiment_df['Review'].apply(lambda x:contractions.fix(str(x)))

#### 2. Lower Casing

In [ ]:
# Lower Casing
sentiment_df['Review']=sentiment_df['Review'].str.lower()

#### 3. Removing Punctuations

In [ ]:
# Remove Punctuations
#function to Remove Punctuations
import string
def remove_punctuation(text):
  '''a function for removing punctuation'''

  # replacing the punctuations with no space,
  # which in effect deletes the punctuation marks
  translator = str.maketrans('', '', string.punctuation)
  # return the text stripped of punctuation marks
  return text.translate(translator)

In [ ]:
#remove punctuation using function created
sentiment_df['Review'] = sentiment_df['Review'].apply(remove_punctuation)
sentiment_df.sample(5)

#### 4. Removing URLs & Removing words and digits contain digits.

In [ ]:
# Remove URLs & Remove words and digits contain
# Remove URLs & Remove words and digits contain digits
import re

# Remove links
sentiment_df["Review"] = sentiment_df["Review"].apply(lambda x: re.sub(r"http\S+", "", x))

# Remove digits
sentiment_df["Review"] = sentiment_df["Review"].apply(lambda x: re.sub(r"\d+", "", x))

#### 5. Removing Stopwords & Removing White spaces

In [ ]:
# Remove Stopwords
# extracting the stopwords from nltk library
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
sw = stopwords.words('english')


In [ ]:
#function to remove stopwords
def delete_stopwords(text):
  '''a function for removing the stopword'''
  # removing the stop words and lowercasing the selected words
  text = [word.lower() for word in text.split() if word.lower() not in sw]
  # joining the list of words with space separator
  return " ".join(text)

In [ ]:
#calling function to remove stopwords
sentiment_df['Review'] = sentiment_df['Review'].apply(delete_stopwords)

In [ ]:
# Remove White spaces
sentiment_df['Review'] =sentiment_df['Review'].apply(lambda x: " ".join(x.split()))

In [ ]:
sentiment_df['Review'][1]

#### 6. Rephrase Text

Not using

In [ ]:
# Rephrase Text
from nltk.corpus import wordnet

#function to create rephrase sentence
def rephrase_sentence(sentence):
     # Tokenize the sentence
     tokens = nltk.word_tokenize(sentence)

     # Replace each token with its synonyms
     new_sentence = []
     for token in tokens:
         synonyms = wordnet.synsets(token)
         if synonyms:
             new_sentence.append(synonyms[0].lemmas()[0].name())
         else:
             new_sentence.append(token)

     # Join the tokens back into a sentence
     rephrased_sentence = " ".join(new_sentence)

     return rephrased_sentence

 # apply the function to the 'Review' column
sentiment_df['Review'] = sentiment_df['Review'].apply(rephrase_sentence)


In [ ]:
review_df['Review'][0]

In [ ]:
sentiment_df['Review'][0]

#### 7. Tokenization

In [ ]:
# Tokenization
sentiment_df['Review'] = sentiment_df['Review'].apply(nltk.word_tokenize)

In [ ]:
sentiment_df['Review'][0]

#### 8. Text Normalization

In [ ]:
# Normalizing Text (i.e., Stemming, Lemmatization etc.)
#applying Lemmatization
from nltk.stem import WordNetLemmatizer

# Create a lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

# Lemmatize the 'Review' column
sentiment_df['Review'] = sentiment_df['Review'].apply(lemmatize_tokens)

##### Which text normalization technique have you used and why?

I have used Stemming technique.It refers to the process of reducing a word to its word stem that affixes to suffixes and prefixes or the roots. While a stemming algorithm is a linguistic normalization process in which the variant forms of a word are reduced to a standard form.
Stemming follows an algorithm with steps to perform on the words which makes it faster. Whereas, in lemmatization, you used a corpus also to supply lemma which makes it slower than stemming.

#### 9. Part of speech tagging

In [ ]:
# POS Taging
#sentiment_df['Review'] = sentiment_df['Review'].apply(nltk.pos_tag)
#sentiment_df.Review

Part-of-speech (POS) tagging can be important for sentiment analysis in some cases, as it can provide additional information about the structure and meaning of the text.

For example, certain POS tags, such as adjectives and adverbs, are often used to express sentiment. By identifying these POS tags in the text, a sentiment analysis model can gain a better understanding of the sentiment being expressed. Additionally, certain grammatical structures, such as negations or modals, can change the sentiment of a sentence. By identifying these structures through POS tagging, a sentiment analysis model can take them into account when determining the overall sentiment of the text.

In [ ]:
sentiment_df.Review[0]

#### 10. Text Vectorization

In [ ]:
# Vectorizing Text
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
vectorizer.fit(sentiment_df['Review'].values)

#creating independent variable for sentiment analysis
X_tfidf = vectorizer.transform(sentiment_df['Review'].values)

In [ ]:
X_tfidf.shape

##### Which text vectorization technique have you used and why?

Here I have used Tf-idf Vectorization technique.

TF-IDF (term frequency-inverse document frequency) is a technique that assigns a weight to each word in a document. It is calculated as the product of the term frequency (tf) and the inverse document frequency (idf).

The term frequency (tf) is the number of times a word appears in a document, while the inverse document frequency (idf) is a measure of how rare a word is across all documents in a collection. The intuition behind tf-idf is that words that appear frequently in a document but not in many documents across the collection are more informative and thus should be given more weight.

The mathematical formula for tf-idf is as follows:

tf-idf(t, d, D) = tf(t, d) * idf(t, D)

where t is a term (word), d is a document, D is a collection of documents, tf(t, d) is the term frequency of t in d, and idf(t, D) is the inverse document frequency of t in D.

The tf component of the weight assigns a value to a word based on how often it appears in the document, while the idf component assigns a value based on how rare the word is in the entire collection of documents. Tf-idf is commonly used in text classification and information retrieval tasks because it can help to down-weight the effect of common words and up-weight the effect of rare words which are more informative.

It also helps to reduce the dimensionality of the data and increases the weight of important words, thus providing more informative and robust feature set for the model to work on.

Text vectorization is the process of converting text data into numerical vectors that can be used as input for machine learning models.

There are several ways to vectorize text data, one of the most common methods is using Tf-idf Vectorization, other methods are bag-of-words (BoW - uses CountVectorizer), word2vec, or doc2vec model.



### 4. Feature Manipulation & Selection

#### 1. Feature Manipulation

In [ ]:
# Manipulate Features to minimize feature correlation and create new features

#Create Total Cuisine Count column based on length of cuisine list
#hotel_cpy['Total_Cuisine_Count'] = hotel_cpy['Cuisines'].apply(lambda x : len(str(x).split(', ')))
#cluster_df=cluster_df.merge(hotel_cpy[['Restaurant','Total_Cuisine_Count']],how="inner",on='Restaurant')

#Find a new column city  fetch from links column
#hotel_cpy['Links']=hotel_cpy.Links.apply(lambda x:x.split('/'))
#hotel_cpy['city']=hotel_cpy.Links.apply(lambda x: x[3])
#hotel_cpy.city.value_counts()

#create a new column in review data set based on rating column
m=sentiment_df.Rating.mean()
sentiment_df['Sentiment']=sentiment_df['Rating'].apply(lambda x: 1 if x>=m else 0)

In [ ]:
cluster_df.shape

I have generated two new columns Total Cuisine count column to find number of cuisines offered by particular restaurant and other one is city column fetched from links column but we can see that all the restaurants are located in Hyderabad except 1.
I have created one new column in the sentiment dataset i.e. Sentiment in which if the rating given by the user is above the avg rating of the whole dataset then its sentiment is 1(positive) otherwise 0(negative).

#### 2. Feature Selection

In [ ]:
# Select your features wisely to avoid overfitting

#Dropping city,Links,Cost_bins columns
hotel_cpy.drop(['city','Links','Cost_bins'],axis=1,inplace=True)

##### What all feature selection methods have you used  and why?

I have dropped city,links and cost bins columns from hotel dataset as these are not useful for further analysis.

##### Which all features you found important and why?

I have chosen those features that help in clustering the restaurant like cuisine,rating,cost per person and total cuisines count.




### 5. Data Transformation

#### Do you think that your data needs to be transformed? If yes, which transformation have you used. Explain Why?

In [ ]:
# Transform Your data
# Getting symmetric and skew symmetric features
symmetric_feature=[]
non_symmetric_feature=[]
for i in cluster_df[['Cost','Rating','Total_Cuisine_Count']]:
  if abs(cluster_df[i].mean()-cluster_df[i].median())<0.1:
    symmetric_feature.append(i)
  else:
    non_symmetric_feature.append(i)

# Getting Symmetric Distributed Features
print("Symmetric Distributed Features : -",symmetric_feature)

# Getting Skew Symmetric Distributed Features
print("Skew Symmetric Distributed Features : -",non_symmetric_feature)

As we can see that only Cost column is skew symmetric so I will apply transformation technique on it to make it normalize.

In [ ]:
#Applying Log transformation on Cost column
cluster_df['Cost']=np.log1p(cluster_df['Cost'])

In [ ]:
#Plot the Cost column after logarithm transformation
plt.figure(figsize = (5,4));
sns.distplot(cluster_df['Cost'], color = '#055E85', fit = norm)
plt.axvline(cluster_df['Cost'].mean(), color='#ff033e', linestyle='dashed', linewidth=3,label= 'mean');  #red
plt.axvline(cluster_df['Cost'].median(), color='#A020F0', linestyle='dashed', linewidth=3,label='median'); #cyan
plt.legend(bbox_to_anchor = (1.0, 1))

After applying log transformation on cost column the feature is following normal distribution.

### 6. Data Scaling

In [ ]:
# Scaling your data
cluster_df.head(3)

As can be seen that Rating,Cost,Total_Cuisine_Count are following symmetric normal distribution but not scaled.

In [ ]:
from sklearn.preprocessing import StandardScaler
#List of features to be scaled
scaled=['Rating','Cost','Total_Cuisine_Count']
scaler=StandardScaler()
scaled_df=pd.DataFrame(scaler.fit_transform(cluster_df[scaled]),columns=scaled)

In [ ]:
#Dropping unscaled features from the cluster data frame
cluster_cpy=cluster_df.copy()
cluster_cpy.drop(['Rating','Cost','Total_Cuisine_Count'],axis=1,inplace=True)
cluster_cpy=pd.concat([cluster_cpy,scaled_df],axis=1)

##### Which method have you used to scale you data and why?

As the data is noramlly distributed so I implemented Standard Scaling.

### 7. Dimesionality Reduction

##### Do you think that dimensionality reduction is needed? Explain Why?

Yes, it is important to use dimensionality reduction techniques as dataset has 40 or more features. This is because, as the number of features increases, the computational cost of clustering algorithms also increases. In addition, high dimensionality can lead to the "curse of dimensionality", where the data becomes sparse and the clusters become harder to identify. Dimensionality reduction techniques such as PCA, t-SNE, or LLE can help reduce the number of features while maintaining the important information in the data, making it easier to cluster and interpret the results.

In [ ]:
# DImensionality Reduction (If needed)
#import PCA
from sklearn.decomposition import PCA

#generating PCA object
pca=PCA()

#fittting the dataframe
cluster_cpy.set_index(['Restaurant'],inplace=True)
pca.fit(cluster_cpy)

In [ ]:
#explained variance v/s no. of components
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker ='o', color = 'orange')
plt.xlabel('number of components',size = 15, color = 'red')
plt.ylabel('cumulative explained variance',size = 14, color = 'blue')
plt.title('Variance v/s No. of Components',size = 20, color = 'green')
plt.xlim([0, 8])
plt.show()

As can be seen that when number of components is 3 it can explain > 60% varaiance.

In [ ]:
pca=PCA(n_components=3)
pca.fit(cluster_cpy)
print(pca.explained_variance_ratio_)
print("Total variance expalained :",np.sum(pca.explained_variance_ratio_))
trans_df=pca.transform(cluster_cpy)

In [ ]:
trans_df.shape

##### Which dimensionality reduction technique have you used and why? (If dimensionality reduction done on dataset.)

I have used PCA as dimensionality reduction technique because PCA (Principal Component Analysis) is a widely used dimensionality reduction technique because it is able to identify patterns in the data that are responsible for the most variation. These patterns, known as principal components, are linear combinations of the original features that are uncorrelated with each other. By using the first few principal components, which account for the majority of the variation in the data, one can effectively reduce the dimensionality of the data while maintaining most of the important information.

Another advantage of PCA is that it is a linear technique, which means it can be applied to data that have a linear relationship between features. It is also easy to interpret the results as the principal components can be thought of as new, uncorrelated features. Additionally, PCA can be used for data visualization by projecting high-dimensional data onto a 2D or 3D space for easy visualization.

When PCA is applied before k-means, it is used to reduce the dimensionality of the data by transforming the original feature space into a new feature space of uncorrelated principal components. The k-means algorithm is then applied to the transformed data, resulting in clusters that are defined in the new feature space. The advantage of this approach is that it can help to remove noise and correlated features from the data, which can make the clustering results more interpretable. However, it also means that the clusters may be harder to interpret in the original feature space.

When PCA is applied after k-means, it is used to visualize the clusters in a lower-dimensional space. The k-means algorithm is applied to the original data, resulting in clusters that are defined in the original feature space. PCA is then used to project the data into a lower-dimensional space, making it easier to visualize and interpret the clusters. The advantage of this approach is that the clusters can be easily interpreted in the original feature space. However, it may not be as effective in removing noise and correlated features from the data.



### 8. Data Splitting

In [ ]:
# Split your data to train and test. Choose Splitting ratio wisely.
X=X_tfidf
y=sentiment_df['Sentiment']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
sentiment_df.shape

In [ ]:
print("X_train:",X_train.shape)
print("Y_train:",y_train.shape)
print("X_test:",X_test.shape)
print("Y_test:",y_test.shape)

##### What data splitting ratio have you used and why?

I have used 80:20 split which is one the most used split ratio. Since there was only 9961 data, therefore I have used more in training set.

### 9. Handling Imbalanced Dataset

##### Do you think the dataset is imbalanced? Explain Why.

In [ ]:
#getting the value count for target class
vc = sentiment_df.Sentiment.value_counts().reset_index().rename(columns =
            {'index':'Sentiment','Sentiment':'Count'})

#defining majority and minority class value
majority_class = vc.Count[0]
minority_class = vc.Count[1]

#calculating cir value for checking class imbalance
CIR = majority_class / minority_class
CIR


In [ ]:
# Dependant Variable Column Visualization
sentiment_df['Sentiment'].value_counts().plot(kind='pie',
                              figsize=(15,6),
                               autopct="%1.1f%%",
                               startangle=90,
                               shadow=True,
                               labels=['Positive Sentiment','Negative Sentiment'],
                               colors=['red','blue'],
                               explode=[0.01,0.02]
                              )
plt.show()

The data is imbalanced as can be seen from the above chart. Here 63% are the postive sentiment while 37% for negavtive sentiment.

There is imbalance in dataset with 60: 40 ratio, where 60 is the majaority class and 40 is the minority class. Even the CIR score suggest that majority class is 1.73 times greater than minority class. However it is considered as slight imbalance, therefore not performing any under or over sampling technique i.e., not required to treat class imabalance.

In [ ]:
# Handling Imbalanced Dataset (If needed)

##### What technique did you use to handle the imbalance dataset and why? (If needed to be balanced)

Answer Here.

## ***7. ML Model Implementation***

### ML Model - 1 KMeans

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Assuming data is your dataset
wcss = []

# Range of cluster numbers to test
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(trans_df)
    wcss.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(8, 4))
plt.plot(k_range, wcss, marker='o', linestyle='-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS')
plt.title('Elbow Method for Optimal k')
plt.grid()
plt.show()


In [ ]:
# ML Model - 1 Implementation
#importing kmeans
from sklearn.cluster import KMeans

# Fit the Algorithm
inertia=[]
for i in range(1,10):
  kmeans=KMeans(n_clusters=i)
  kmeans.fit(trans_df)
  inertia.append(kmeans.inertia_)


In [ ]:
sns.lineplot(y=inertia,x=range(1,10),marker='o',linestyle='-')
plt.grid()
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm

In [ ]:
#silhouette score
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
from sklearn.model_selection import ParameterGrid
# candidates for the number of cluster
parameters = list(range(2,10))
#parameters
parameter_grid = ParameterGrid({'n_clusters': parameters})
best_score = -1
#visualizing Silhouette Score for individual clusters and the clusters made
for n_clusters in parameters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # 1st subplot is the silhouette plot
    # silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(trans_df) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(trans_df)

    # silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(trans_df, cluster_labels)
    print("For n_clusters =", n_clusters,
          "average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(trans_df, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("silhouette plot for the various clusters.")
    ax1.set_xlabel("silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(trans_df[:, 0], trans_df[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')
    #marker='' % i will give numer in cluster in 2 plot
    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")
    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

In [ ]:
#vizualizing the clusters and the datapoints in each clusters
plt.figure(figsize = (10,6), dpi = 120)

kmeans= KMeans(n_clusters = 6, init= 'k-means++', random_state = 42)
kmeans.fit(trans_df)

#predict the labels of clusters.
label = kmeans.fit_predict(trans_df)
#Getting unique labels
unique_labels = np.unique(label)

#plotting the results:
for i in unique_labels:
    plt.scatter(trans_df[label == i , 0] , trans_df[label == i , 1] , label = i)
plt.legend()
plt.show()

In [ ]:
#Creating a new column Label and assign cluster labels to it
cluster_df['Label']=label

In [ ]:
#changing back cost value to original from log1p done during transformation
cluster_df['Cost'] = np.expm1(cluster_df['Cost'])

#Merging the cluster data frame with the hotel to add relevalent columns
cluster_df=cluster_df[['Restaurant','Cost','Rating','Total_Cuisine_Count','Label']].merge(hotel_cpy[['Restaurant','Cuisines']],on='Restaurant')
cluster_df.head(3)

In [ ]:
#find the number of restaurants in each cluster
cluster_df.groupby('Label')['Restaurant'].count().sort_values().reset_index()

In [ ]:
#Find the cuisines for each cluster
#create a copy of cluster data frame and seperate rows for each cuisine
new_cluster_df=cluster_df.copy()
new_cluster_df['Cuisines']=cluster_df.Cuisines.str.split(',')
new_cluster_df=new_cluster_df.explode('Cuisines',ignore_index=True)

In [ ]:
#group cluster by label and find list of cuisines
grouped_cluster=new_cluster_df.groupby('Label')
for label,group in grouped_cluster:
  print(f'Cuisines for cluster {label}')
  print(group['Cuisines'].tolist())
  print()

Agglomerative Hierarchical Clustering
Hierarchial clustering algorithms group similar objects into groups called clusters. There are two types of hierarchical clustering algorithms:

Agglomerative — Bottom up approach. Start with many small clusters and merge them together to create bigger clusters. Divisive — Top down approach. Start with a single cluster than break it up into smaller clusters.

Agglomerative hierarchical clustering

The agglomerative hierarchical clustering algorithm is a popular example of HCA. To group the datasets into clusters, it follows the bottom-up approach. It means, this algorithm considers each dataset as a single cluster at the beginning, and then start combining the closest pair of clusters together. It does this until all the clusters are merged into a single cluster that contains all the datasets. This hierarchy of clusters is represented in the form of the dendrogram.

Dendrogram in Hierarchical clustering

The dendrogram is a tree-like structure that is mainly used to store each step as a memory that the HC algorithm performs. In the dendrogram plot, the Y-axis shows the Euclidean distances between the data points, and the x-axis shows all the data points of the given dataset.



In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
# Perform hierarchical clustering
linked = linkage(trans_df, 'ward')

# Create a dendrogram
plt.figure(figsize=(10, 5))
dendrogram(linked, orientation='top', labels=range(1, len(trans_df) + 1), distance_sort='descending', show_leaf_counts=True)

plt.title('Dendrogram')
plt.xlabel('Restaurants')
plt.ylabel('Distance')
plt.show()

In [ ]:
#Checking the Silhouette score for 15 clusters
from sklearn.cluster import AgglomerativeClustering

range_n_clusters = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]
for n_clusters in range_n_clusters:
    hc = AgglomerativeClustering(n_clusters = n_clusters, affinity = 'euclidean', linkage = 'ward')
    y_hc = hc.fit_predict(trans_df)
    score = silhouette_score(trans_df, y_hc)
    print("For n_clusters = {}, silhouette score is {}".format(n_clusters, score))

In [ ]:
# agglomerative clustering
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification

# define the model
model = AgglomerativeClustering(n_clusters = 6)      #n_clusters=6
# fit model and predict clusters
y_hc = model.fit_predict(trans_df)
# retrieve unique clusters
clusters = unique(y_hc)
# create scatter plot for samples from each cluster
for cluster in clusters:
	# get row indexes for samples with this cluster
	row_ix = where(y_hc == cluster)
	# create scatter of these samples
	plt.scatter(trans_df[row_ix, 0], trans_df[row_ix, 1])
# show the plot
plt.show()
#Evaluation

#Silhouette Coefficient
print("Silhouette Coefficient: %0.3f"%silhouette_score(trans_df,y_hc, metric='euclidean'))

#davies_bouldin_score of our clusters
from sklearn.metrics import davies_bouldin_score
davies_bouldin_score(trans_df, y_hc)
print("davies_bouldin_score %0.3f"%davies_bouldin_score(trans_df, y_hc))

In [ ]:
#creating new colummn for predicting cluster using hierarcial clsutering
cluster_df['Label_hr'] = y_hc

In [ ]:
cluster_df.head()

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

KMeans Clustering

I applied K means Clustering to cluster the Restaurants based on the given features. I used both the Elbow and Silhuoette Methods to get an efficient number of K, and we discovered that n clusters = 6 was best for our model. The model was then fitted using K means, and each data point was labelled with the cluster to which it belonged using K means.labels. After labelling the clusters, we visualised them and counted the number of restaurants in each cluster, discovering that the majority of the restaurants belonged to the fourth cluster.

Agglomerative Hierarchical Clustering

I have used Hierarchial Clustering - Agglomerative Model to cluster the restaurants based on different features. This model uses a down-top approach to cluster the data. I have used Silhouette Coefficient Score and used clusters = 6 and then vizualized the clusters and the datapoints within it.



In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

Not Needed

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### ML Model - 2 Sentiment Analysis

####Unsupervised Sentiment Analysis Using LDA

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score
silhouette_scores = []
for n_components in range(2,11):
    lda = LatentDirichletAllocation (n_components=n_components)
    lda.fit(X)
    labels = lda.transform(X).argmax(axis=1)
    silhouette_scores.append(silhouette_score(X, labels))

In [ ]:
#plotting silhouette score
plt.plot(range(2,11), silhouette_scores, marker ='o', color='red')
plt.xlabel('Number of Topics', size = 15, color = 'green')
plt.ylabel('Silhouette Score', size = 15, color = 'blue')
plt.grid()
plt.show()

In [ ]:
# LDA model
lda = LatentDirichletAllocation(n_components=3)
lda.fit(X)

In [ ]:
#creating copy to store predicted sentiments
review_sentiment_prediction = review_cpy.copy()
review_sentiment_prediction.head()

In [ ]:
# predicting the sentiments and storing in a feature
topic_results = lda.transform(X)
review_sentiment_prediction['Prediction'] = topic_results.argmax(axis=1)
review_sentiment_prediction.sample(5)

In [ ]:
print(lda.components_)

In [ ]:
#worcloud
# Define the number of words to include in the word cloud
N = 100

# Create a list of strings for each topic
topic_text = []
for index, topic in enumerate(lda.components_):
    topic_words = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-N:]]
    print(topic_words)
    #topic_text.append(" ".join(topic_words))

# Create a word cloud for each topic
#for i in range(len(topic_text)):
#    print(f'TOP 100 WORDS FOR TOPIC #{i}')
#    wordcloud = WordCloud(background_color="black",colormap='rainbow').generate(topic_text[i])
#    plt.figure(figsize=(10,5))
#    plt.imshow(wordcloud, interpolation='bilinear')
#    plt.axis("off")
#    plt.show()
#    print('='*120)

In [ ]:
#using pyldavis to visualise
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

Answer Here.

### ML Model - 3

In [ ]:
# ML Model - 3 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 3 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

Answer Here.

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

Answer Here.

### 3. Explain the model which you have used and the feature importance using any model explainability tool?

Answer Here.

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***